In [1]:
import pandas as pd
import numpy as np
import requests, json
import sys
import os



In [2]:
client_id = os.environ['BNET_CLIENT_ID']
client_secret = os.environ['BNET_SECRET']

client_id = os.environ['BNET_CLIENT_ID']
client_secret = os.environ['BNET_SECRET']

def create_access_token(client_id, client_secret, region = 'us'):
    data = { 'grant_type': 'client_credentials' }
    response = requests.post('https://%s.battle.net/oauth/token' % region, data=data, auth=(client_id, client_secret))
    return response.json()

token = create_access_token(client_id, client_secret)['access_token']